Процесс обучения и оценки некоторых моделей я отдельно загружать не буду. Только пропишу их результаты тут:

Model v1.1 $\\$ 
epoch3_data3.pth $\\$ 
recall: 0.823 $\\$ 
precision: 0.6 $\\$ 
f1: 0.692 $\\$ 

Model v2.0 $\\$ 
989_256_49_epoch№.pth $\\$
Epoch 1 $\\$ 
recall: 0.854 $\\$
precision: 1.0 $\\$
f1: 0.727 $\\$ 


Epoch 2 $\\$ 
recall: 0.877 $\\$
precision: 0.75 $\\$ 
f1: 0.773 $\\$


Epoch 3 $\\$ 
recall: 0.867 $\\$ 
precision: 0.6 $\\$
f1: 0.764 $\\$ 


Epoch 4 $\\$ 
recall: 0.865 $\\$
precision: 0.75 $\\$
f1: 0.75 $\\$


Model v3.0 $\\$
989_x2_256_49_epoch№.pth $\\$
Epoch 1 $\\$
recall: 0.849 $\\$
precision: 0.75 $\\$
f1: 0.721 $\\$


Epoch 2 $\\$
recall: 0.868 $\\$
precision: 1.0 $\\$
f1: 0.767 $\\$


Epoch 3 $\\$
recall: 0.864 $\\$
precision: 0.75 $\\$
f1: 0.764 $\\$




In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_pickle("/kaggle/input/data0-25/data_norm0.25.pkl")
df2 = pd.read_pickle("/kaggle/input/data1-0/data_norm1.pkl")
df3 = pd.read_pickle("/kaggle/input/data0-5/data_norm0.5.pkl")

In [3]:
df = pd.concat([df1, df2])
df = pd.concat([df, df3])

In [4]:
df = df.reset_index() 

In [5]:
df

,index,OUTPUT,INPUT
0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1242904446...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0770836309982...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[0, 0, 0, 0, 0, 0.10748772366243657, 0, 0, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0.06577925136276556, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
1292574,599995,"[0, 0, 0.23119837918566855, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292575,599996,"[0, 0, 0, 0, 0, 0, 0, 0.07482285803921773, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292576,599997,"[0, 0, 0.13130826532079368, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292577,599998,"[0.0866654566110813, 0.0684285664685669, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
del df1
del df2
del df3

In [7]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data.iloc[idx, 0], dtype=torch.float)
        y = torch.tensor(self.data.iloc[idx, 1], dtype=torch.float)
        return x, y

dataset = CustomDataset(df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [8]:
import torch
import torch.nn as nn


class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(989, 989)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(989, 256)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(256, 49)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

model = CustomModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

In [11]:
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

Evaluate

In [10]:
models_list = ["/kaggle/input/version2/pytorch/989_256_49/1/989_256_49_epoch1.pth", 
              "/kaggle/input/version2/pytorch/989_256_49/1/989_256_49_epoch2.pth",
              "/kaggle/input/version2/pytorch/989_256_49/1/989_256_49_epoch3.pth", 
              "/kaggle/input/version2/pytorch/989_256_49/1/989_256_49_epoch4.pth"]

In [13]:
import numpy
from sklearn.metrics import precision_score, recall_score, f1_score

for i in models_list:
    model = torch.load(i)
    recall_list = []
    f1_list = []
    precision_list = []
    for targets, inputs in dataloader:
        out = model.forward(inputs).detach().numpy()
        targets = targets.detach().numpy()
        out = np.argmax(out, axis=1)
        targets = np.argmax(targets, axis=1)

        precision = precision_score(out, targets, average=None, zero_division=1)
        recall = recall_score(out, targets, average=None, zero_division=1)
        f1 = f1_score(out, targets, average=None, zero_division=1)

    
        precision_list.extend(precision)
        recall_list.extend(recall)
        f1_list.extend(f1)
    
    print(i.split("/")[-1])
    print("recall:", sum(recall_list)/len(recall_list))
    print("precision:", sum(precision)/len(precision))
    print("f1:", sum(f1_list)/len(f1_list))

989_256_49_epoch1.pth
recall: 0.8541554866131599
precision: 1.0
f1: 0.727009045361947
989_256_49_epoch2.pth
recall: 0.8765324109687705
precision: 0.75
f1: 0.7726994717641346
989_256_49_epoch3.pth
recall: 0.8668862489131851
precision: 0.6
f1: 0.7640020011913391
989_256_49_epoch4.pth
recall: 0.8647130309287978
precision: 0.75
f1: 0.7509622985744111
